In [37]:
from pystac.client import Client
from odc.stac import load
from odc.geo.geom import BoundingBox
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon

In [38]:
# DEP STAC catalog
# catalog = "https://stac.staging.digitalearthpacific.io"
catalog = "https://stac.prod.digitalearthpacific.io"

# SDB collection 
collection = "dep_s2_sdb" 

In [48]:
## Makogai
#bbox = BoundingBox(178.888, -17.535, 179.008, -17.393)
#bbox.explore()

#Suva
# bbox = BoundingBox(178.39155, -18.19648, 178.54308, -18.09867)
# bbox.explore()

#Solomon Isl
bbox = BoundingBox(159.660, -8.500, 159.683, -8.485)
#bbox = gpd.read_file("AOI_Bathymetry_Fiona.geojson")
bbox.explore()

In [49]:
client = Client.open(catalog)

items = client.search(
    collections=[collection], bbox=bbox, datetime="2024"
).item_collection()

print(f"Found {len(items)} items in the collection {collection}.")

Found 1 items in the collection dep_s2_sdb.


In [50]:
# Load the DEP data
data = load(
    items,
    bbox=bbox,
    bands=["depth"],
    chunks={}
)

data

<xarray.Dataset> Size: 177kB
Dimensions:      (y: 169, x: 257, time: 1)
Coordinates:
  * y            (y) float64 1kB -9.417e+05 -9.417e+05 ... -9.434e+05 -9.434e+05
  * x            (x) float64 2kB 1.075e+06 1.075e+06 ... 1.078e+06 1.078e+06
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables:
    depth        (time, y, x) float32 174kB dask.array<chunksize=(1, 169, 257), meta=np.ndarray>

In [51]:
import folium
from ipyleaflet import basemaps

centre = (bbox.bottom + bbox.top) / 2, (bbox.left + bbox.right) / 2
map = folium.Map(location=centre, zoom_start=12, tiles=basemaps.Esri.WorldImagery)
data.depth.odc.add_to(map)

map

In [52]:
data.depth.odc.write_cog("sol_depth.tif", overwrite=True)

PosixPath('sol_depth.tif')